In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import Adagrad
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [5]:
dataset = pd.read_csv("data.csv")

In [6]:
dataset.shape
dataset.head()

Unnamed: 0   X1   X2   X3   X4   X5   X6   X7   X8   X9  ...  X170  X171  \
0  X21.V1.791  135  190  229  223  192  125   55   -9  -33  ...   -17   -15   
1  X15.V1.924  386  382  356  331  320  315  307  272  244  ...   164   150   
2     X8.V1.1  -32  -39  -47  -37  -32  -36  -57  -73  -85  ...    57    64   
3   X16.V1.60 -105 -101  -96  -92  -89  -95 -102 -100  -87  ...   -82   -81   
4   X20.V1.54   -9  -65  -98 -102  -78  -48  -16    0  -21  ...     4     2   

   X172  X173  X174  X175  X176  X177  X178  y  
0   -31   -77  -103  -127  -116   -83   -51  4  
1   146   152   157   156   154   143   129  1  
2    48    19   -12   -30   -35   -35   -36  5  
3   -80   -77   -85   -77   -72   -69   -65  5  
4   -12   -32   -41   -65   -83   -89   -73  5  

[5 rows x 180 columns]

In [7]:
X = dataset.drop(columns=['Unnamed: 0', 'y'], axis=1)
X.head()

X1   X2   X3   X4   X5   X6   X7   X8   X9  X10  ...  X169  X170  X171  \
0  135  190  229  223  192  125   55   -9  -33  -38  ...     8   -17   -15   
1  386  382  356  331  320  315  307  272  244  232  ...   168   164   150   
2  -32  -39  -47  -37  -32  -36  -57  -73  -85  -94  ...    29    57    64   
3 -105 -101  -96  -92  -89  -95 -102 -100  -87  -79  ...   -80   -82   -81   
4   -9  -65  -98 -102  -78  -48  -16    0  -21  -59  ...    10     4     2   

   X172  X173  X174  X175  X176  X177  X178  
0   -31   -77  -103  -127  -116   -83   -51  
1   146   152   157   156   154   143   129  
2    48    19   -12   -30   -35   -35   -36  
3   -80   -77   -85   -77   -72   -69   -65  
4   -12   -32   -41   -65   -83   -89   -73  

[5 rows x 178 columns]

In [8]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X)

C:\Users\HCL\AppData\Local\Continuum\anaconda4\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


[[ 0.88505134  1.20992878  1.46276429 ... -0.63414367 -0.43329036
  -0.23539922]
 [ 2.40057718  2.36619038  2.23944096 ...  1.02342937  0.95424076
   0.85653664]
 [-0.12328657 -0.16915405 -0.22513147 ... -0.13687176 -0.13859348
  -0.14440456]
 ...
 [ 0.1544592   0.10184476 -0.01720228 ...  0.0657205   0.07015014
   0.02545213]
 [-0.17159018 -0.08484331  0.00725997 ...  0.49546166  0.43852123
   0.40762968]
 [ 0.24502848  0.31262161  0.41088722 ...  0.0657205   0.08856869
   0.19530882]]


C:\Users\HCL\AppData\Local\Continuum\anaconda4\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [9]:
Y = dataset['y']
Y.head()

0    4
1    1
2    5
3    5
4    5
Name: y, dtype: int64

In [10]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
print(encoded_Y)

[3 0 4 ... 4 2 3]


In [11]:
categorical_Y = np_utils.to_categorical(encoded_Y)
print(categorical_Y)

[[0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [12]:
def baseline_model():
    model = Sequential()
    model.add(Dense(1024, input_dim=178, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1024, activation='sigmoid'))
    model.add(Dropout(0.4))
    model.add(Dense(5, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adagrad(), metrics=['accuracy'])
    return model

In [13]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=50, batch_size=200, verbose=0)

In [14]:
kfold = KFold(n_splits=5, shuffle=True, random_state=3)

In [15]:
results = cross_val_score(estimator, X, categorical_Y, cv=kfold)
print('Accuracy: {}'.format(results.mean()))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Accuracy: 0.8927129149436951
